# Inference: Adaptive Covariance MCMC

This example shows you how to perform Bayesian inference on a time series, using [Adaptive Covariance MCMC](http://pints.readthedocs.io/en/latest/mcmc_samplers/adaptive_covariance_mcmc.html).

It follows on from the [first sampling example](./sampling-first-example.ipynb).

In [1]:
import os
os.chdir('C:/Users/bclamber/Desktop/AllPints/pints4/pints')
from __future__ import print_function
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.UnknownNoiseLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01, 400, noise*0.1],
    [0.02, 600, noise*100]
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
xs = [
    real_parameters * 1.1,
    real_parameters * 0.9,
    real_parameters * 1.15,
]

# Create mcmc routine
mcmc = pints.MCMCSampling(log_posterior, 3, xs, method=pints.DramMCMC)

# Add stopping criterion
mcmc.set_max_iterations(4000)

# Start adapting after 1000 iterations
mcmc.set_initial_phase_iterations(1000)

# Disable logging mode
mcmc.set_log_to_screen(False)

# Run!
print('Running...')
chains = mcmc.run()
print('Done!')

# Show traces and histograms
pints.plot.trace(chains)

# Discard warm up
chains = chains[:, 2000:, :]

# Check convergence using rhat criterion
print('R-hat:')
print(pints.rhat_all_params(chains))

# Look at distribution in chain 0
pints.plot.pairwise(chains[0], kde=True)

# Show graphs
plt.show()

Running...
-3
<class 'numpy.ndarray'>
[[  1.65000000e-02   5.50000000e+02   1.10000000e+01]
 [  1.35000000e-02   4.50000000e+02   9.00000000e+00]
 [  1.72500000e-02   5.75000000e+02   1.15000000e+01]]
(3, 3)
[[  1.65000000e-02   5.50000000e+02   1.10000000e+01]
 [  1.35000000e-02   4.50000000e+02   9.00000000e+00]
 [  1.72500000e-02   5.75000000e+02   1.15000000e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.51683047e-02   5.46842956e+02   1.06773558e+01]
 [  1.35000000e-02   4.50000000e+02   9.00000000e+00]
 [  1.72500000e-02   5.75000000e+02   1.15000000e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.51683047e-02   5.46842956e+02   1.06773558e+01]
 [  1.35000000e-02   4.50000000e+02   9.00000000e+00]
 [  1.72500000e-02   5.75000000e+02   1.15000000e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.51683047e-02   5.46842956e+02   1.06773558e+01]
 [  1.60035625e-02   4.49761940e+02   8.80867529e+00]
 [  1.72500000e-02   5.75000000e+02   1.15000000e+01]]
<class 'numpy.ndarray'>
(3, 3)
-3
<class 'numpy

(3, 3)
[[  1.45644861e-02   5.45159851e+02   1.04712955e+01]
 [  1.58842028e-02   4.51196085e+02   8.63051873e+00]
 [  1.43097619e-02   5.72725347e+02   1.16597075e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.45644861e-02   5.45159851e+02   1.04712955e+01]
 [  1.58842028e-02   4.51196085e+02   8.63051873e+00]
 [  1.43097619e-02   5.72725347e+02   1.16597075e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.45644861e-02   5.45159851e+02   1.04712955e+01]
 [  1.58842028e-02   4.51196085e+02   8.63051873e+00]
 [  1.43097619e-02   5.72725347e+02   1.16597075e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.45644861e-02   5.45159851e+02   1.04712955e+01]
 [  1.58842028e-02   4.51196085e+02   8.63051873e+00]
 [  1.43097619e-02   5.72725347e+02   1.16597075e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.45644861e-02   5.45159851e+02   1.04712955e+01]
 [  1.58842028e-02   4.51196085e+02   8.63051873e+00]
 [  1.43097619e-02   5.72725347e+02   1.16597075e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.45644861e-02   5.

 [  1.39573359e-02   5.92924048e+02   3.00477829e+01]]
<class 'numpy.ndarray'>
(3, 3)
-3
<class 'numpy.ndarray'>
[[  1.49335205e-02   4.99924561e+02   8.33243054e+00]
 [  1.52873759e-02   4.90321213e+02   1.33851735e+01]
 [  1.40517090e-02   5.97198541e+02   3.39984173e+01]]
(3, 3)
[[  1.49335205e-02   4.99924561e+02   8.33243054e+00]
 [  1.50198165e-02   4.97265716e+02   1.42212126e+01]
 [  1.40517090e-02   5.97198541e+02   3.39984173e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49335205e-02   4.99924561e+02   8.33243054e+00]
 [  1.50198165e-02   4.97265716e+02   1.42212126e+01]
 [  1.40517090e-02   5.97198541e+02   3.39984173e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49277055e-02   5.01203931e+02   8.39310974e+00]
 [  1.50198165e-02   4.97265716e+02   1.42212126e+01]
 [  1.40517090e-02   5.97198541e+02   3.39984173e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49277055e-02   5.01203931e+02   8.39310974e+00]
 [  1.50198165e-02   4.97265716e+02   1.42212126e+01]
 [  1.40517090e-02   5.9719

<class 'numpy.ndarray'>
(3, 3)
[[  1.49799534e-02   4.99606366e+02   8.31571956e+00]
 [  1.49952653e-02   4.98046695e+02   1.43149505e+01]
 [  1.40710345e-02   5.99413514e+02   3.60312417e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49799534e-02   4.99606366e+02   8.31571956e+00]
 [  1.49952653e-02   4.98046695e+02   1.43149505e+01]
 [  1.40710345e-02   5.99413514e+02   3.60312417e+01]]
<class 'numpy.ndarray'>
(3, 3)
-3
<class 'numpy.ndarray'>
[[  1.49799534e-02   4.99606366e+02   8.31571956e+00]
 [  1.49952653e-02   4.98046695e+02   1.43149505e+01]
 [  1.40710345e-02   5.99413514e+02   3.60312417e+01]]
(3, 3)
[[  1.49799534e-02   4.99606366e+02   8.31571956e+00]
 [  1.49952653e-02   4.98046695e+02   1.43149505e+01]
 [  1.39793422e-02   5.99558031e+02   3.61331328e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49799534e-02   4.99606366e+02   8.31571956e+00]
 [  1.49952653e-02   4.98046695e+02   1.43149505e+01]
 [  1.39793422e-02   5.99558031e+02   3.61331328e+01]]
<class 'numpy.ndarray'>
(3,

 [  1.36275438e-02   5.99989514e+02   3.64160266e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50043516e-02   5.00318659e+02   8.34848794e+00]
 [  1.50082640e-02   4.98003969e+02   1.43107524e+01]
 [  1.36275438e-02   5.99989514e+02   3.64160266e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50091704e-02   5.00359869e+02   8.35006322e+00]
 [  1.50171428e-02   4.97612897e+02   1.42627548e+01]
 [  1.36275438e-02   5.99989514e+02   3.64160266e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50034754e-02   5.00526572e+02   8.35827989e+00]
 [  1.50388265e-02   4.97155473e+02   1.42094674e+01]
 [  1.36275438e-02   5.99989514e+02   3.64160266e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50031640e-02   5.00573713e+02   8.36049020e+00]
 [  1.50434368e-02   4.96995447e+02   1.41903465e+01]
 [  1.36275438e-02   5.99989514e+02   3.64160266e+01]]
<class 'numpy.ndarray'>
(3, 3)
-3
<class 'numpy.ndarray'>
[[  1.50031640e-02   5.00573713e+02   8.36049020e+00]
 [  1.50434368e-02   4.96995447e+02   1.41903465e+01]
 [ 

[[  1.49873994e-02   5.00759345e+02   8.38297139e+00]
 [  1.49855066e-02   4.97606745e+02   1.42423193e+01]
 [  1.36628185e-02   5.99977623e+02   3.64157928e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50003126e-02   5.00142424e+02   8.34717013e+00]
 [  1.49855066e-02   4.97606745e+02   1.42423193e+01]
 [  1.36628185e-02   5.99977623e+02   3.64157928e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50124396e-02   5.00367159e+02   8.36794019e+00]
 [  1.49850094e-02   4.98289255e+02   1.43442397e+01]
 [  1.36260108e-02   5.99991955e+02   3.64170155e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50133467e-02   5.00491502e+02   8.36385090e+00]
 [  1.50310015e-02   4.96931469e+02   1.41895128e+01]
 [  1.36260108e-02   5.99991955e+02   3.64170155e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49630129e-02   5.01238094e+02   8.37758472e+00]
 [  1.50147522e-02   4.98509384e+02   1.43638709e+01]
 [  1.36260108e-02   5.99991955e+02   3.64170155e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49734891e-02   5.0100441

 [  1.36469700e-02   5.99966003e+02   3.64015131e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50128548e-02   4.98803703e+02   1.03015803e+01]
 [  1.49135570e-02   5.03831494e+02   1.22453900e+01]
 [  1.36469700e-02   5.99966003e+02   3.64015131e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49986575e-02   4.99444982e+02   1.06292652e+01]
 [  1.49135570e-02   5.03831494e+02   1.22453900e+01]
 [  1.36469700e-02   5.99966003e+02   3.64015131e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49926743e-02   5.00178869e+02   9.78949391e+00]
 [  1.49135570e-02   5.03831494e+02   1.22453900e+01]
 [  1.36469700e-02   5.99966003e+02   3.64015131e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49926743e-02   5.00178869e+02   9.78949391e+00]
 [  1.49135570e-02   5.03831494e+02   1.22453900e+01]
 [  1.36170656e-02   5.99998013e+02   3.64220593e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.49926743e-02   5.00178869e+02   9.78949391e+00]
 [  1.49516633e-02   5.03034136e+02   1.22611159e+01]
 [  1.36170656e-02   5.9999801

<class 'numpy.ndarray'>
(3, 3)
[[  1.50209213e-02   4.99692864e+02   1.05778927e+01]
 [  1.49686708e-02   5.00336512e+02   1.02108047e+01]
 [  1.37151569e-02   5.99976304e+02   3.64339142e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50209213e-02   4.99692864e+02   1.05778927e+01]
 [  1.49686708e-02   5.00336512e+02   1.02108047e+01]
 [  1.37234721e-02   5.99992261e+02   3.64528888e+01]]
<class 'numpy.ndarray'>
(3, 3)
-3
<class 'numpy.ndarray'>
[[  1.50209213e-02   4.99692864e+02   1.05778927e+01]
 [  1.49686708e-02   5.00336512e+02   1.02108047e+01]
 [  1.37335562e-02   5.99976739e+02   3.64439246e+01]]
(3, 3)
[[  1.50209213e-02   4.99692864e+02   1.05778927e+01]
 [  1.49658572e-02   5.00388666e+02   1.07148708e+01]
 [  1.37223115e-02   5.99958198e+02   3.64227428e+01]]
<class 'numpy.ndarray'>
(3, 3)
[[  1.50209213e-02   4.99692864e+02   1.05778927e+01]
 [  1.49855735e-02   5.00735051e+02   1.04182825e+01]
 [  1.37046138e-02   5.99906148e+02   3.63734268e+01]]
<class 'numpy.ndarray'>
(3,

C:\Users\bclamber\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3146: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\bclamber\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\bclamber\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\bclamber\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bclamber\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: float division by zero

In [3]:
chains

array([], shape=(3, 0, 3), dtype=float64)